*Арешин Станислав Олегович*

**Лабораторная работа №1**

**Моделирование работы системы массового обслуживания**

# Постановка задачи

**Задание:**

*Есть парикмахерская, с $n$ парикмахерами и очередью. Предполагается, что поток клиентов, желающих постричься – пуассоновский с интенсивностью $\lambda$, поток постриженных $i$-м парикмахером клиентов – пуассоновский с интенсивностью $\mu_{i}$.*

*Выбрав значения $n$, $\lambda$ и $\mu_{i}$ , написать программу, моделирующую работу этой парикмахерской в течение рабочего дня с 10:00 до 20:00.*

# Реализация

Для хранения очереди клиентов выбран и реализован в виде класса контейнер Queue.

In [1]:
class Queue():
    def __init__(self):
        self.items = []
        
    def push(self, item):
        self.items.append(item)
        
    def pop(self):
        return self.items.pop(0)
    
    def __len__(self):
        return len(self.items)
    
    def show(self):
        return self.items

Далее реализован класс Hairdressing_Salon, имитирующий работу парикмахерской

In [2]:
import numpy as np
from scipy.stats import poisson
import pandas as pd
import random

class Hairdressing_Salon():
    
    def __init__(self, num_hairdressers, lambd, mu, time_period, start_time_work = 0):
        self.start_time_work = start_time_work # время начала работы
        self.lambd = lambd # интенсивность потока клиентов  
        self.mu = mu # интенсивность потока работы парикмахера
        self.time_period = time_period # время работы
        #self.mx = 1 / lambd # среднее время ожидания заявок на обслуживание
        self.my = [1 / mu[i] for i in range(num_hairdressers)]  # среднее время обработки 1 заявки 1 парикмахером
        self.num_clients = self.num_clients()
        self.num_hairdressers = num_hairdressers
        self.hairdressers = [0 for i in range(num_hairdressers)] # список парикмахеров, 0-свободен, 1 - занят
        self.clients = Queue()
        self.result = [] # результат работы
        self.time_to_free = [self.start_time_work for i in range(self.num_hairdressers)]
        
    # время ожидания следующего клиента
    def client_next_time(self):
        time = random.expovariate(self.lambd)
        return round(time)
    
    # кол-во клиентов за весь промежуток времени
    def num_clients(self):
        return poisson.rvs(self.time_period * self.lambd, size=1)[0]
    
    # время работы парикмахера с одним клиентов
    def hairdresser_work_time(self, hairdresser_i):
        time_work = random.expovariate(self.mu[hairdresser_i])
            
        return round(time_work)
    
    # забираем клиента из очереди 
    def take_from_queue(self, time):
        
        # флаг, если в этот момент времени освободится парикмахер
        flag = 0
        # проходим по всем паркмахерам и смотрим, освободились ли они в этот момент времени
        for i in range (self.num_hairdressers):
            if (self.start_time_work + time) >= self.time_to_free[i]:
                # если был занят, а должен быть свободен - flag = 1
                if self.hairdressers[i] == 1:
                    flag = 1
                # и освобождаем
                self.hairdressers[i] = 0
            else:
                self.hairdressers[i] = 1
                
        # если парикмахер освободился        
        if flag == 1:
            # сохраняем результат
            self.result.append([self.start_time_work + time,[elem for elem in self.hairdressers], len(self.clients)])        
        # смотрим, какие парикмахеры свободны, список индексов
        free_hairdressers = [i for i in range(self.num_hairdressers) if  self.hairdressers[i] == 0]
        
        # если есть свободные паркмахеры и есть клиенты в очереди
        if len(self.clients) > 0 and len(free_hairdressers) != 0:
            # удаляем клиента из очереди
            self.clients.pop()  
            # выбираем случайного свободного паркмахера, индекс
            free_hairdresser = free_hairdressers[np.random.choice(len(free_hairdressers), 1)[0]]
            # делаем отметку парикмахер занят
            self.hairdressers[free_hairdresser] = 1
            # делаем отметку о начале времени работы
            self.time_to_free[free_hairdresser] = self.start_time_work
            self.time_to_free[free_hairdresser] += time
            # сохраняем результаты о начале работы
            self.result.append([self.start_time_work + time,[elem for elem in self.hairdressers], len(self.clients)])
            # время, которое парикмахер работает 
            time_work = self.hairdresser_work_time(free_hairdresser)
            # делаем отметку о  конце времени работы
            self.time_to_free[free_hairdresser] += time_work
            
    # добавление клиента
    def add_client(self, time, client):
        
        # флаг, если в этот момент времени освободится парикмахер
        flag = 0
        # проходим по всем паркмахерам и смотрим, освободились ли они в этот момент времени
        for i in range (self.num_hairdressers):
            if (self.start_time_work + time) >= self.time_to_free[i]:
                # если был занят, а должен быть свободен - flag = 1
                if self.hairdressers[i] == 1:
                    flag = 1
                # и освобождаем
                self.hairdressers[i] = 0
            else:
                self.hairdressers[i] = 1
                
        # если парикмахер освободился          
        if flag == 1:
            # сохраняем результат
            self.result.append([self.start_time_work + time,[elem for elem in self.hairdressers], len(self.clients)])  
        # смотрим, какие парикмахеры свободны, список индексов       
        free_hairdressers = [i for i in range(self.num_hairdressers) if  self.hairdressers[i] == 0]
    
        # если свободных нет, добавляем в очередь
        if len(free_hairdressers) == 0:
            self.clients.push(client)
            self.result.append([self.start_time_work + time,[elem for elem in self.hairdressers], len(self.clients)])
            
        # иначе случайный свободный парикмахер начинает работу
        else:
            # выбираем случайного свободного паркмахера, индекс
            free_hairdresser = free_hairdressers[np.random.choice(len(free_hairdressers), 1)[0]]
            # делаем отметку парикмахер занят
            self.hairdressers[free_hairdresser] = 1
            # делаем отметку о начале времени работы
            self.time_to_free[free_hairdresser] = self.start_time_work
            self.time_to_free[free_hairdresser] += time
            # сохраняем результаты
            self.result.append([self.start_time_work + time,[elem for elem in self.hairdressers], len(self.clients)])
            # время, которое парикмахер работает 
            time_work = self.hairdresser_work_time(free_hairdresser)
            # делаем отметку о  конце времени работы
            self.time_to_free[free_hairdresser] += time_work
                  
    # рассчёт времени прихода клиентов        
    def get_coming_time(self):
        time = 0
        coming_time = []
        
        for _ in range(self.num_clients):
            time_wait = self.client_next_time()
            time += time_wait
            coming_time.append(time)
            
        return coming_time
    
    # отображение результатов
    def show_result(self):
        res = pd.DataFrame(self.result, columns= ['Время', 'Парикмахеры', 'Очередь'])
        time = res['Время'].copy().to_list()
        hours_start = 10
        minutes_start = 0
        
        for i in range (len(time)):
            hours = time[i] // 60
            minutes = time[i] % 60
            if (hours_start + hours) // 10 !=0:
                if minutes // 10 != 0:
                    time[i] = f'{hours_start + hours}:{minutes}'
                else: 
                    time[i] = f'{hours_start + hours}:0{minutes}'
            else:
                if  minutes // 10 != 0:
                    time[i] = f'0{hours_start + hours}:{minutes}'
                else: 
                    time[i] = f'0{hours_start + hours}:0{minutes}'
        
        res['Время'] = time
        res = res.set_index('Время') 
        return res 
    
    # запуск работы парикмахерской                      
    def run(self):
        # рассчитываем время прихода клиентов
        coming_time = self.get_coming_time()
        
        # проходим по каждому временному промежутку по минуте от первого до последнего прихода
        for time in np.arange(coming_time[0], coming_time[len(coming_time)- 1]+1, 1):
            # если в это время пришёл клиент, то добавляем его
            if time in coming_time:
                self.add_client(time, 1)
            # в противном случае пробуем взять клиента из очереди
            else:
                self.take_from_queue(time)
                
        # если клиенты в очереди остались, обслуживаем их 
        time = coming_time[len(coming_time)- 1]
        while(len(self.clients)!= 0):
            time += 1
            self.take_from_queue(time)
            
        # обслуживаем последних клиентов
        while(True):
            time += 1
            self.take_from_queue(time)
            free_hairdressers = [i for i in range(self.num_hairdressers) if  self.hairdressers[i] == 0]
            if len(free_hairdressers) == self.num_hairdressers:
                break
                
        return self.show_result()

# Тесты

Для тестов я зафиксировал параметр $\lambda = 0.2$ - интенсивность потока клиентов и параметр $n$ = 3 - кол-во парикмахеров.

Период работы -  time_period выбран 8 часов (480 минут)

Меняя парметры интесивности обработки 1 клиента 1 парикмахером - $\mu_{i}$ посмотрим, что получается.

In [3]:
model = Hairdressing_Salon(3,0.2,[0.1,0.05, 0.1],time_period=480)
res = model.run()
pd.set_option('display.max_rows', res.shape[0]+1)
res

,Парикмахеры,Очередь
Время,,
10:01,"[0, 1, 0]",0
10:07,"[0, 0, 0]",0
10:09,"[0, 0, 1]",0
10:13,"[0, 0, 0]",0
10:14,"[0, 1, 0]",0
10:22,"[0, 1, 1]",0
10:23,"[0, 1, 0]",0
10:28,"[0, 1, 1]",0
10:29,"[0, 0, 1]",0


In [4]:
# 480 - время с 10 до 18
model = Hairdressing_Salon(3,0.2,[0.1,0.05, 0.2],time_period=480)
res = model.run()
pd.set_option('display.max_rows', res.shape[0]+1)
res

,Парикмахеры,Очередь
Время,,
10:11,"[0, 1, 0]",0
10:13,"[1, 1, 0]",0
10:22,"[1, 1, 1]",0
10:23,"[1, 0, 0]",0
10:23,"[1, 1, 0]",0
10:30,"[1, 1, 1]",0
10:32,"[1, 1, 0]",0
10:34,"[1, 1, 1]",0
10:36,"[1, 0, 0]",0


In [6]:
model = Hairdressing_Salon(3,0.2,[0.1,0.15, 0.2],time_period=480)
res = model.run()
pd.set_option('display.max_rows', res.shape[0]+1)
res

,Парикмахеры,Очередь
Время,,
10:12,"[1, 0, 0]",0
10:18,"[1, 1, 0]",0
10:21,"[1, 0, 0]",0
10:35,"[1, 1, 0]",0
10:38,"[1, 1, 1]",0
10:39,"[1, 0, 1]",0
10:41,"[1, 0, 0]",0
10:43,"[0, 0, 0]",0
10:43,"[1, 0, 0]",0


In [8]:
model = Hairdressing_Salon(3,0.2,[0.2,0.2,0.2],time_period=480)
res = model.run()
pd.set_option('display.max_rows', res.shape[0]+1)
res

,Парикмахеры,Очередь
Время,,
10:04,"[0, 0, 1]",0
10:05,"[0, 0, 0]",0
10:06,"[0, 0, 1]",0
10:11,"[0, 0, 0]",0
10:11,"[0, 0, 1]",0
10:14,"[0, 0, 0]",0
10:15,"[1, 0, 0]",0
10:17,"[1, 0, 1]",0
10:20,"[1, 0, 0]",0


## Вывод

При интесивности работы парикмахеров заметно ниже, чем интесивность потока клиентов наблюдается скопление очереди более 20 человек и за счёт этого время работы приходится увеличивать на несколько часов, чтобы обработать всю очередь.

Если же приравнять эти параметры, очередь практически не скапливается, парикмахерская закрывается раньше срока.

Для $\lambda = 0.2$ и $n$ = 3 оптимальным значением $\mu_{i}$ будет 0.1, 0.05, 0.2. Тогда парикмахерская закрывается в срок.

